In [33]:
# 필사 : Stacking Test-Sklearn, XGBoost, CatBoost, LightGBM
# https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
# from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

NFOLDS = 3
SEED = 0
NROWS = None

data = pd.read_csv('./data/application_train.csv')
test = pd.read_csv('./data/application_test.csv')
prev = pd.read_csv('./data/previous_application.csv')


In [35]:
print(data.shape)
data.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
print(test.shape)
test.head()

(48744, 121)


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

for f_ in categorical_feats :
    data[f_], indexer = pd.factorize(data[f_]) # 라벨인코딩해줌
    test[f_] = indexer.get_indexer(test[f_])

In [38]:
gc.enable()
y_train = data['TARGET']
del data['TARGET']

In [39]:
prev_cat_features = [
    f for f in prev if prev[f].dtype == 'object'
]

for f_ in prev_cat_features :
    prev[f_], _ = pd.factorize(prev[f_])

avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [40]:
x_train = data.merge(right = avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

In [41]:
class SklearnWrapper(object) :
    def __init__(self, clf, seed=0, params=None) :
        params['random_state']=seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train) :
        self.clf.fit(x_train, y_train)
        
    def predict(self, x) :
        return self.clf.predict_proba(x)[:,1]
    
class CatboostWrapper(object) :
    def __init__(self, clf, seed=0, params=None) :
        params['random_seed']=seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x) :
        return self.clf.predict_proba(x)[:,1]

class LightGBMWrapper(object):
    def __init__(self,clf,seed=0,params=None):
        params['feature_fraction_seed']=seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x) :
        return self.clf.predict_proba(x)[:,1]
    
class XgbWrapper(object) :
    def __init__(self, seed=0, params=None) :
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)
        
    def train(self, x_train, y_train) :
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)
        
    def predict(self, x) :
        return self.gbdt.predict(xgb.DMatrix(x))
# oof : out of fold    
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)) :
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]
        
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te) 
        # 각각 validation set 평가 -> 다합치면 전체 training set에 대한 예측
        oof_test_skf[i, :] = clf.predict(x_test) # test set 으로 예측
        
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1), oof_test.reshape(-1,1)
    


In [42]:
et_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' :2
}

rf_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.2,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

xgb_params = {
    'seed' : 0,
    'colsample_bytree' : 0.7,
    'silent' : 1,
    'subsample' : 0.7,
    'learning_rate' : 0.075,
    'objective' : 'binary:logistic',
    'max_depth' : 4,
    'num_parallel_tree' : 1,
    'min_child_weight' : 1,
    'nrounds' : 200,
    'verbosity' : 0
}

catboost_params = {
    'iterations' : 200,
    'learning_rate' : 0.5,
    'depth' : 3,
    'l2_leaf_reg' : 40,
    'bootstrap_type' : 'Bernoulli',
    'subsample' : 0.7,
    'scale_pos_weight' : 5,
    'eval_metric' : 'AUC',
    'od_type' : 'Iter',
    'allow_writing_files' : False,
    'verbose' : False
}

lightgbm_params = {
    'n_estimators' : 200,
    'learning_rate' : 0.1,
    'num_leaves' : 123,
    'colsample_bytree' : 0.8,
    'subsample' : 0.9,
    'max_depth' : 15,
    'reg_alpha' : 0.1,
    'reg_lambda' : 0.1,
    'min_split_gain' : 0.01,
    'min_child_weight' : 2
}

In [43]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf = ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf = RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf = CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed=SEED, params = lightgbm_params)

In [44]:
xg_oof_train, xg_oof_test = get_oof(xg)
print('XG-CV : {}'.format(sqrt(mean_squared_error(y_train, xg_oof_train))))


XG-CV : 0.25969546149690725


In [45]:
et_oof_train, et_oof_test = get_oof(et)
print('ET-CV : {}'.format(sqrt(mean_squared_error(y_train, et_oof_train))))


ET-CV : 0.26296502347137407


In [46]:
rf_oof_train, rf_oof_test = get_oof(rf)
print('RF-CV : {}'.format(sqrt(mean_squared_error(y_train, rf_oof_train))))


RF-CV : 0.2629856213006646


In [47]:
cb_oof_train, cb_oof_test = get_oof(cb)
print('CB-CV : {}'.format(sqrt(mean_squared_error(y_train, cb_oof_train))))

CB-CV : 0.3309876006961808


In [48]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

print('{}, {}'.format(x_train.shape, x_test.shape))

(307511, 4), (48744, 4)


In [49]:
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]
test
# test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,100001,0,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.050847
1,100005,0,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.130254
2,100013,0,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0,0.025552
3,100028,0,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0.039884
4,100038,0,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.147140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,1,0,0,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.038672
48740,456222,0,1,0,1,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.078578
48741,456223,0,1,1,0,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0,0.030688
48742,456224,0,0,0,1,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0.052314
